---
# Transformer XL
---

---
# Import Libraries

In [1]:
import os
import sys

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
sys.path.append("../../../deep-learning-dna")
sys.path.append("../")
sys.path.append("../../../deep-learning-dna/common")

In [4]:
import wandb

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import math
import string

from Attention import Set_Transformer
from common.models import dnabert
from common import dna
from lmdbm import Lmdb
from common.data import DnaSequenceGenerator, DnaLabelType, DnaSampleGenerator, find_dbs
import wandb

from core.custom_objects import CustomObject

import tf_utils as tfu

In [6]:
strategy = tfu.devices.select_gpu(0, use_dynamic_memory=True)

---
# Load Data

In [7]:
#Import pretrained model
api = wandb.Api()
model_path = api.artifact("sirdavidludwig/dnabert-pretrain/dnabert-pretrain-8dim:latest").download()
pretrained_model = dnabert.DnaBertModel.load(model_path)
pretrained_model.load_weights(model_path + "/model.h5")
pretrained_model

In [8]:
#Load datafiles
dataset_path = api.artifact("sirdavidludwig/nachusa-dna/dnasamples-complete:latest").download('/data/dna_samples:v1')
samples = find_dbs(dataset_path + '/train')
samples[13]

wandb: Downloading large artifact dnasamples-complete:latest, 4079.09MB. 420 files... Done. 0:0:0.1


'/data/dna_samples:v1/train/WS-CCW-Jul2015_S82_L001_R1_001.db'

---
# Create Dataset

In [9]:
split_ratios = [0.8, 0.2]
set_len = 1000
sequence_len = 150
kmer = 3
batch_size = [20,5]
batches_per_epoch = 20
augument = True
labels = DnaLabelType.SampleIds
seed = 0
rng = np.random.default_rng(seed)
random_samples = samples.copy()

In [10]:
rng.shuffle(random_samples)

In [11]:
trimmed_samples, (train_dataset, val_dataset) = DnaSampleGenerator.split(samples=random_samples[0:5], split_ratios=split_ratios, subsample_length=set_len, sequence_length=sequence_len, kmer=kmer,batch_size=batch_size,batches_per_epoch=batches_per_epoch,augment=augument,labels=labels, rng=rng)

In [12]:
random_samples[0:5]

['/data/dna_samples:v1/train/WS-CCE-Apr2016_S6_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes52-10-TC_S53_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-WH-Jul2016_S46_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes41-10-HN_S42_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wesley026-Ag-072820_S165_L001_R1_001.db']

--- 
# Batch Parameters

In [13]:
seg_size = 200
max_set_len = set_len
max_files = len(train_dataset.samples)
max_files

5

In [14]:
if seg_size-2 > set_len:
    raise ValueError("Segment size should not be bigger than sequence length")

In [15]:
print(max_set_len)

1000


---
# Create Embeddings

In [16]:
#Create 8 dimensional embeddings
pretrained_encoder = dnabert.DnaBertEncoderModel(pretrained_model.base)
pretrained_encoder.trainable = False

In [17]:
class Create_Embeddings(keras.layers.Layer):
    def __init__(self, encoder):
        super(Create_Embeddings, self).__init__()
        self.encoder = encoder
        
    def subbatch_predict(self, model, batch, subbatch_size, concat=lambda old, new: tf.concat((old, new), axis=0)):
        def predict(i, result=None):
            n = i + subbatch_size
            pred = tf.stop_gradient(model(batch[i:n]))
            if result is None:
                return [n, pred]
            return [n, concat(result, pred)]
        i, result = predict(0)
        batch_size = tf.shape(batch)[0]
        
        i, result = tf.while_loop(
            cond=lambda i, _: i < batch_size,
            body=predict,
            loop_vars=[i, result],
            parallel_iterations=1)
        return result
    
    
    def modify_data_for_input(self, data):
        batch_size = tf.shape(data)[0]
        subsample_size = tf.shape(data)[1]
        flat_data = tf.reshape(data, (batch_size*subsample_size, -1))
        encoded = self.subbatch_predict(self.encoder, flat_data, 128)
        return tf.reshape(encoded, (batch_size, subsample_size, -1))
    
    def call(self, data):
        return  self.modify_data_for_input(data)

---
# Cache Memory

In [18]:
def Cache_Memory(current_state, previous_state, memory_length):
    if memory_length is None or memory_length == 0:
        return None
    else:
        if previous_state is None:
            new_mem = current_state[:, -memory_length:, :]
        else:
            new_mem = tf.concat(
                    [previous_state, current_state], 1)[:, -memory_length:, :]

    return tf.stop_gradient(new_mem)

---
# Attention

In [19]:
class Attention(keras.Model):
    def __init__(self, num_induce, embed_dim, num_heads, use_layernorm, pre_layernorm, use_keras_mha):
        super(Attention, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        if self.num_induce == 0:       
            self.attention = (Set_Transformer.SetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm,use_keras_mha=self.use_keras_mha))
        else:
            self.attention = Set_Transformer.InducedSetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, num_induce=self.num_induce, use_layernorm=self.use_layernorm, pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha)
    
    
    def call(self, data, mems):
                
            attention = self.attention([data, mems])
                
            return attention

---
# XL Block

In [20]:
class TransformerXLBlock(tf.keras.layers.Layer):
    def __init__(self,
                 num_induce, 
                 embed_dim,
                 num_heads,
                 use_layernorm,
                 pre_layernorm,
                 use_keras_mha,):

        super(TransformerXLBlock, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.attention = Attention
        
        self.attention_layer = self.attention(self.num_induce, self.embed_dim, self.num_heads, self.use_layernorm, self.pre_layernorm, self.use_keras_mha)

   
    def call(self,
             content_stream,
             state=None):
        
        attention_output = self.attention_layer(content_stream, state)

        return attention_output

---
# Transformer XL

In [21]:
class TransformerXL(tf.keras.layers.Layer):
    def __init__(self,
                 mem_switched, 
                 num_layers,
                 num_induce,
                 embed_dim,
                 num_heads,
                 dropout_rate,
                 mem_len=None,
                 use_layernorm=True,
                 pre_layernorm=True, 
                 use_keras_mha=True):
        
        super(TransformerXL, self).__init__()

        self.mem_switched = mem_switched
        self.num_layers = num_layers
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        self.mem_len = mem_len
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha

        self.transformer_xl_layers = []
        
        for i in range(self.num_layers):
            self.transformer_xl_layers.append(
                    TransformerXLBlock(self.num_induce,
                                        self.embed_dim,
                                        self.num_heads,
                                        self.use_layernorm,
                                        self.pre_layernorm, 
                                        self.use_keras_mha))

        self.output_dropout = tf.keras.layers.Dropout(rate=self.dropout_rate)

    def call(self,
             content_stream,
             state=None):
        
        new_mems = []

        if state is None:
            state = [None] * self.num_layers
            
        for i in range(self.num_layers):
            if self.mem_switched == False:
                new_mems.append(Cache_Memory(content_stream, state[i], self.mem_len))
            
            transformer_xl_layer = self.transformer_xl_layers[i]
            
            transformer_xl_output = transformer_xl_layer(content_stream=content_stream,
                                                        state=state[i])
            
            content_stream = self.output_dropout(transformer_xl_output)
            
            if self.mem_switched == True:
                new_mems.append(Cache_Memory(content_stream, state[i], self.mem_len))

        output_stream = content_stream
        return output_stream, new_mems

---
# Xl Model Class

In [61]:
class XlModel(keras.Model):
    def __init__(self, mem_switched, max_files, encoder, seg_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, mem_len, dropout_rate, num_seeds, use_layernorm, pre_layernorm, use_keras_mha):
        super(XlModel, self).__init__()
        
        self.mem_switched = mem_switched
        self.max_files = max_files
        self.encoder = encoder
        self.seg_size = seg_size
        self.max_set_len = max_set_len
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.mem_len = mem_len
        self.dropout_rate = dropout_rate
        self.num_seeds = num_seeds
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.embedding_layer = Create_Embeddings(self.encoder)

        self.linear_layer = keras.layers.Dense(self.embed_dim)
        
        self.transformer_xl = TransformerXL(self.mem_switched,
                                            self.num_layers,
                                             self.num_induce,
                                             self.embed_dim,
                                             self.num_heads,
                                             self.dropout_rate,
                                             self.mem_len,
                                             self.use_layernorm,
                                             self.pre_layernorm,
                                             self.use_keras_mha)
        

        self.pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=self.num_seeds,embed_dim=self.embed_dim,num_heads=self.num_heads,use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha, is_final_block=True)
    
        self.reshape_layer = keras.layers.Reshape((self.embed_dim,))
   
        self.output_layer = keras.layers.Dense(self.max_files, activation=keras.activations.softmax)
        
    
    def call(self, x, mems, index, training=None):        
        
        #Embed full set
        embeddings = self.embedding_layer(x)
            
        #Linearly transform full set
        linear_transform = self.linear_layer(embeddings)
        
        #Pull out current segment
        segment = linear_transform[:, index:index+self.seg_size]
        
        output, mems = self.transformer_xl(content_stream=linear_transform, state=mems)
                
        pooling = self.pooling_layer(output)

        reshape = self.reshape_layer(pooling)

        output = self.output_layer(reshape)          
        
        return output, mems

---
# Xl Parameters

In [62]:
#Xl Parameters
encoder = pretrained_encoder
mem_switched=False
num_induce = 0
embed_dim = 64
num_layers = 8
num_heads = 8
mem_len = 200
dropout_rate = 0.01
num_seeds = 1
use_layernorm = True
pre_layernorm = True
use_keras_mha = True

In [63]:
Parameters = dict(mem_switched=False,
                max_files=max_files,
                seg_size=seg_size,
                max_set_len=max_set_len,
                num_induce = 0,
                embed_dim = 64,
                num_layers = 8,
                num_heads = 8,
                mem_len = 200,
                dropout_rate = 0.01,
                num_seeds = 1,
                use_layernorm = True,
                pre_layernorm = True,
                use_keras_mha = True)

In [64]:
run = wandb.init(project="Test", config=Parameters)

---
# Create Models

In [65]:
model = XlModel(mem_switched, max_files, encoder, seg_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, mem_len, dropout_rate, num_seeds, use_layernorm, pre_layernorm, use_keras_mha)
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer = keras.optimizers.Adam(1e-3), metrics = keras.metrics.SparseCategoricalAccuracy())

In [66]:
epochs=2

In [67]:
loss_function = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
accuracy_function = keras.metrics.SparseCategoricalAccuracy()

In [97]:
subbatch_size = 1

In [ ]:
def accumulate_train_step(train_step, batch, subbatch_size, model):
    
    def step(i, accum_variables=None, accum_grads=None):
        n = i + subbatch_size
        subbatch = (batch[0][i:n], batch[1][i:n])
        
        variables, grads = train_step(subbatch)
        
        if accum_variables is not None:
            variables = accumulate(variables, accum_variables)
            
        grads = [[(g + ag) for g, ag in zip(gs, ags)] for gs, ags in zip(grads, accum_grads)]    
        return [n, variables, grads]
    
    accum_grads = [tf.zeros_like(w) for w in model.trainable_weights]
    i, variables, accum_grads = step(0, None, accum_grads)
    
    batch_size = tf.shape(batch[0])[0]
    
    i, variables, accum_grads = tf.while_loop(
        cond=lambda i, *_: i < batch_size,
        body=step,
        loop_vars=[i, variables, accum_grads],
        parallel_iterations=1)
    
    return variables, accum_grads

In [43]:
@tf.function()
def train_step(batch):
    
    def step(subbatch):
        x, y = subbatch
        
        with tf.GradientTape() as tape:
            predicted = model(x, training=True)
            loss = loss_function(y, predicted)
            accuracy = accuracy_function(y, predicted)
            
        grads = tape.gradient(loss, model.trainable_weights)
        
        return [], [grads] 
    
    variables, (grads,) = accumulate_train_step(step, batch, subbatch_size, model)
    
    model.optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
    return loss, accuracy

In [ ]:
@tf.function()
def train_step(inputs):
    
    batch, max_set_len, seg_size = inputs
    
    #Iterate through subbatches
    #Pull out one set at a time
    for i in range (batch_size[0]):
        n = i + subbatch_size
        one_set = (batch[0][i:n], batch[1][i:n])
        x, y = one_set
        i += 1
        
        #Initialize accumulated gradients
        accumulated_grads = [tf.zeros_like(w) for w in model.trainable_weights]
        
        #Initialize mems
        mems = tf.zeros((self.num_layers, tf.shape(x)[0], self.mem_len, self.embed_dim))
        
        #Split set into segments
        for index in range(0, max_set_len, seg_size):
            
            #Pass entire set and memories into model
            with tf.GradientTape() as tape:
                segment_output, mems = model(x, mems, index, training=True)
                
                loss = loss_function(y, segment_output)
                accuracy = accuracy_function(y, segment_output)

            #Compute segment level gradients
            grads = tape.gradient(loss, model.trainable_weights)
                
            #Add gradients to accumulated gradients
            accumulated_grads = [[(g + ag) for g, ag in zip(gs, ags)] for gs, ags in zip(grads, accum_grads)]    
    
            #Apply gradients
            model.optimizer.apply_gradients(zip(accumulated_grads, model.trainable_weights))

    return loss, accuracy

In [88]:
for epoch in range(epochs):    
    
    train_loss = 0
    train_accuracy = 0
    val_loss= 0
    val_accuracy = 0
    
    #Iterate through batches
    for batch in train_dataset:
        
        #Pass one batch intro train_step
        loss, accuracy = train_step([batch, max_set_len, seg_size])
        
        train_loss += loss
        train_accuracy += accuracy
    
    train_loss = train_loss / batch_size
    train_accuracy = train_accuracy / batch_size

    print(f"\r{epoch+1}/{epochs} Train Loss: {train_loss} Train Accuracy = {train_accuracy}) # Val Loss: {val_loss} Val Accuracy = {val_accuracy}", end="")

NameError: name 'train_step' is not defined

In [ ]:
history = model.fit(x=train_dataset, validation_data=val_dataset, epochs=epochs, verbose=1, callbacks=[wandb.keras.WandbCallback(save_weights_only=True)])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING Unable to compute FLOPs for this model.


Epoch 1/2
 2/20 [==>...........................] - ETA: 1:21 - loss: 2.2917 - sparse_categorical_accuracy: 0.3000

In [ ]:
run.finish()

In [ ]:
model.save_weights("./Saved_Models/StrXL_Run2.h5")